In [52]:
import cvxpy as cp
import numpy as np

# decision 
a = 1.4  # a와 b는 주어지는 상수로 설정할 수 있습니다.
b = 0.7
Premise_var_x1 = np.array([[0, a/2, a]]) # >= 0 value 만 setting 음수는 의미 없음
Premise_var_x3 = np.array([[-b, -b/2, b/2, b]]) # b는 양수, 음수 모두 중요함

n_r = np.size(Premise_var_x1)*np.size(Premise_var_x3)  # number of rules
n_s = 4  # A{1}의 열 크기 (고정된 크기)
n_u = 1  # B{1}의 열 크기 (고정된 크기)

# A와 B 행렬 정의
A = []
B = []

A = []
for i in range(np.size(Premise_var_x1)):
  for j in range(np.size(Premise_var_x3)):
    x1 = Premise_var_x1[0,i]
    x3 = Premise_var_x3[0,j]

    A13 = np.sin(x3)/x3
    A31 = x1**2
    A43 = np.sin(x3)/x3
    B11 = x1**2
    A.append(np.array([[1, 1, A13, -0.1],
                   [1, -2, 0, 0],
                   [1, A31, -0.3, 0],
                   [0, 0, A43, -1]]))
    B.append(np.array([[1+B11],
                   [0],
                   [0],
                   [0]]))

# SDP 변수 정의
Q = cp.Variable((n_s, n_s), symmetric=True)
Y = [cp.Variable((n_u, n_s)) for _ in range(n_r)]
I = np.identity(n_s)
#gamma = cp.Variable((1, 1))
# LMI constraints
constraints = []


In [83]:

# LMI_total_c1 constraints (모든 i에 대해)
for i in range(n_r):
    Ai = A[i]
    Bi = B[i]
    constraints.append((Ai @ Q + Q @ Ai.T + Bi @ Y[i] + Y[i].T @ Bi.T) << 0)

# LMI_total_c2 constraints
# 첫 번째 그룹 (LMI_total_c2_1)
for j in range(1, n_r):
    Ai = A[0]
    Aj = A[j]
    Bi = B[0]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[0] + Y[0].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# # 두 번째 그룹 (LMI_total_c2_2)
for j in range(2, n_r):
    Ai = A[1]
    Aj = A[j]
    Bi = B[1]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[1] + Y[1].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# # 세 번째 그룹 (LMI_total_c2_3)
for j in range(3, n_r):
    Ai = A[2]
    Aj = A[j]
    Bi = B[2]
    Bj = B[j]
    constraint_lhs = (Ai @ Q + Q @ Ai.T + Bi @ Y[j] + Y[j].T @ Bi.T) / 2
    constraint_rhs = (Aj @ Q + Q @ Aj.T + Bj @ Y[2] + Y[2].T @ Bj.T) / 2
    constraints.append(constraint_lhs + constraint_rhs << 0)

# Positive semidefinite constraint on Q
constraints.append(Q - 0.1*I>> 0)
#constraints.append(gamma >> 0)
# SDP 문제 정의 및 해결
problem = cp.Problem(cp.Minimize(0), constraints)
print(problem.is_qp())
problem.solve(verbose = True)  # 여기서 SCS나 다른 solver를 선택할 수 있습니다.

# # 최적해 계산
ch = [c.dual_value for c in constraints]

K = [Y[i].value @ np.linalg.inv(Q.value) for i in range(n_r)]

for i in range(n_r):
  print(f'K{{{i+1}}}=', K[i])


True
                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Aug 11 01:46:10 PM: Your problem has 64 variables, 8944 constraints, and 0 parameters.
(CVXPY) Aug 11 01:46:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 11 01:46:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 11 01:46:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 11 01:46:10 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 11 01:46:10 PM: Compiling problem (target solver=SCS

{}
